In [1]:
!pip install statsforecast

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import matplotlib.pyplot as plt
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import OneHotEncoder
from pylab import rcParams
import os
  
# Ignore harmless warnings 
import warnings 
warnings.filterwarnings("ignore") 


StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 59.7 MB/s eta 0:00:00


In [2]:
# Load dataset
konya = pd.read_parquet("abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/konya_features.parquet")

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 4, Finished, Available, Finished)

In [3]:
# vizualize dataset
konya.head()
konya.tail()

# print info
print(konya.info())

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 5, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 32 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   store_id               532614 non-null  object        
 1   product_id             532614 non-null  object        
 2   date                   532614 non-null  datetime64[ns]
 3   sales                  532614 non-null  float64       
 4   revenue                532614 non-null  float64       
 5   stock                  532614 non-null  float64       
 6   price                  531602 non-null  float64       
 7   promo_type_1           532614 non-null  category      
 8   promo_bin_1            75780 non-null   category      
 9   promo_type_2           532614 non-null  category      
 10  promo_bin_2            479 non-null     category      
 11  promo_discount_2       479 non-null     float64       
 12  promo_discount_type_2  479 non-null     cate

In [4]:
# change dtypes 

cat_col = ['store_id', 'product_id', 'season', 'week']
konya[cat_col] = konya[cat_col].astype('category')

# Drop columns not necessary
df = konya.copy()
# List of columns to drop
columns_to_drop = [
    'promo_bin_1', 
    'promo_bin_2',
    'revenue', 
    'promo_discount_2', 
    'promo_discount_type_2', 
    'hierarchy2_id', 
    'hierarchy3_id', 
    'hierarchy4_id', 
    'hierarchy5_id', 
    'city_id_old', 
    'country_id', 
    'city_code',
    'product_length', 
    'product_depth',
    'product_width',
    'weekday'
]

# Drop the specified columns
df = df.drop(columns=columns_to_drop, errors='ignore')

#confirm drop
df.info()

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 6, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 532614 entries, 0 to 532613
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   store_id       532614 non-null  category      
 1   product_id     532614 non-null  category      
 2   date           532614 non-null  datetime64[ns]
 3   sales          532614 non-null  float64       
 4   stock          532614 non-null  float64       
 5   price          531602 non-null  float64       
 6   promo_type_1   532614 non-null  category      
 7   promo_type_2   532614 non-null  category      
 8   cluster_id     532614 non-null  category      
 9   hierarchy1_id  532614 non-null  category      
 10  storetype_id   532614 non-null  category      
 11  store_size     532614 non-null  int64         
 12  season         532614 non-null  category      
 13  week           532614 non-null  category      
 14  holiday        532614 non-null  category      
 15  

In [5]:
# separate products that are not sold in 2019

# Ensure 'date' column is of datetime type
df['date'] = pd.to_datetime(df['date'])

# Filter products that are present in 2019
products_in_2019 = df[df['date'].dt.year == 2019]['product_id'].unique()

# Filter X_ts based on products in 2019
df_filtered = df[df['product_id'].isin(products_in_2019)]

# Example: Display the filtered DataFrame
print(df_filtered.head(20))
print(df['product_id'].nunique())
print(df_filtered['product_id'].nunique())

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 7, Finished, Available, Finished)

   store_id product_id       date  sales  stock  price promo_type_1  \
0     S0030      P0015 2017-01-02    0.0    4.0   2.60         PR14   
1     S0030      P0018 2017-01-02    1.0    5.0   1.95         PR14   
2     S0030      P0035 2017-01-02    2.0    1.0   2.45         PR14   
3     S0030      P0051 2017-01-02    0.0   27.0   0.70         PR14   
4     S0030      P0055 2017-01-02    0.0   12.0   3.50         PR05   
5     S0030      P0057 2017-01-02    0.0    4.0  12.90         PR14   
6     S0030      P0062 2017-01-02    0.0    5.0  19.90         PR14   
8     S0030      P0103 2017-01-02    0.0   13.0   2.65         PR14   
9     S0030      P0114 2017-01-02    1.0   20.0   0.45         PR14   
10    S0030      P0116 2017-01-02    3.0   16.0   0.85         PR14   
11    S0030      P0125 2017-01-02    0.0    9.0   2.45         PR14   
12    S0030      P0131 2017-01-02    0.0    9.0   2.25         PR14   
13    S0030      P0171 2017-01-02    0.0    9.0   9.85         PR14   
14    

In [6]:
# Get unique store_ids
unique_store_ids = df_filtered['store_id'].unique()

# Create a dictionary of DataFrames, each keyed by store_id
store_datasets = {}
for store_id in unique_store_ids:
    store_datasets[store_id] = df_filtered[df_filtered['store_id'] == store_id].copy()

# dataset for store_id 'S0142'
store_S0142	 = store_datasets['S0142']
print("Store S0142 dataset:")
print(store_S0142.head())

# dataset for store_id S0094
store_S0094	 = store_datasets['S0094']
print("Store S0094 dataset:")
print(store_S0094.head())

# dataset for store_id S0030
store_S0030	 = store_datasets['S0030']
print("Store S0030 dataset:")
print(store_S0030.head())


StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 8, Finished, Available, Finished)

Store S0142 dataset:
       store_id product_id       date  sales  stock  price promo_type_1  \
329162    S0142      P0005 2017-01-02    0.0    2.0  33.90         PR14   
329163    S0142      P0015 2017-01-02    0.0   12.0   2.60         PR14   
329164    S0142      P0017 2017-01-02    0.0   17.0   1.49         PR14   
329165    S0142      P0018 2017-01-02    0.0    2.0   1.95         PR14   
329168    S0142      P0051 2017-01-02    0.0   15.0   0.70         PR14   

       promo_type_2 cluster_id hierarchy1_id storetype_id  store_size season  \
329162         PR03  cluster_9           H03         ST04          31      1   
329163         PR03  cluster_1           H00         ST04          31      1   
329164         PR03  cluster_6           H00         ST04          31      1   
329165         PR03  cluster_4           H00         ST04          31      1   
329168         PR03  cluster_7           H00         ST04          31      1   

       week holiday month_name  
329162    1   

In [7]:
# select store as df
df = store_S0094.copy() # replace according to store

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 9, Finished, Available, Finished)

In [8]:
# Change the granularity of time series data from daily to weekly

# Set 'ds' column as the index
df.set_index('date', inplace=True)
# Sort the DataFrame by the index (dates)
df.sort_index(inplace=True)

# Group by 'product_id' and resample to weekly frequency
df_week = df.groupby(['product_id']).resample('W-MON').agg({
    'sales': 'sum',  # Aggregate sales data
    'stock': 'last',  # Take the last stock value of the week
    'price': 'last',  # Last price for the week
    'promo_type_1': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'promo_type_2': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'cluster_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'hierarchy1_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'storetype_id': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'store_size': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'season': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'holiday': lambda x: x.mode().iloc[0] if not x.mode().empty else None, 
    'week': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
    'month_name': lambda x: x.mode().iloc[0] if not x.mode().empty else None,  # Mode of categorical variable
}).reset_index()

# Display the weekly aggregated DataFrame
print(df_week.head())
print(df_week.tail())


df_week.info()

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 10, Finished, Available, Finished)

  product_id       date  sales  stock  price promo_type_1 promo_type_2  \
0      P0005 2018-01-15    0.0    6.0   39.9         PR14         PR03   
1      P0005 2018-01-22    0.0    6.0   39.9         PR14         PR03   
2      P0005 2018-01-29    0.0    6.0   39.9         PR14         PR03   
3      P0005 2018-02-05    0.0    6.0   39.9         PR14         PR03   
4      P0005 2018-02-12    0.0    6.0   39.9         PR14         PR03   

  cluster_id hierarchy1_id storetype_id  store_size  season holiday  week  \
0  cluster_9           H03         ST04        45.0     1.0       N   2.0   
1  cluster_9           H03         ST04        45.0     1.0       N   3.0   
2  cluster_9           H03         ST04        45.0     1.0       N   4.0   
3  cluster_9           H03         ST04        45.0     1.0       N   5.0   
4  cluster_9           H03         ST04        45.0     1.0       N   6.0   

  month_name  
0        Jan  
1        Jan  
2        Jan  
3        Feb  
4        Feb  
  

# **ARIMA**

# **TENTATIVA com exogenas**

In [9]:
# filter one product (top 3 sold)

#select top 3
unique_ids = ['P0103', 'P0569', 'P0131']

# Filter the DataFrame based on the list of unique IDs
df_select = df_week[df_week['product_id'].isin(unique_ids)].copy()

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 11, Finished, Available, Finished)

In [10]:
df_select.info()

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 12, Finished, Available, Finished)

<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 4592 to 30220
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_id     432 non-null    category      
 1   date           432 non-null    datetime64[ns]
 2   sales          432 non-null    float64       
 3   stock          431 non-null    float64       
 4   price          431 non-null    float64       
 5   promo_type_1   431 non-null    object        
 6   promo_type_2   431 non-null    object        
 7   cluster_id     431 non-null    object        
 8   hierarchy1_id  431 non-null    object        
 9   storetype_id   431 non-null    object        
 10  store_size     431 non-null    float64       
 11  season         431 non-null    float64       
 12  holiday        431 non-null    object        
 13  week           431 non-null    float64       
 14  month_name     431 non-null    object        
dtypes: category(1), datetim

In [11]:
# check missing values
missing_values_count = df_select.isna().sum()
missing_values_count

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 13, Finished, Available, Finished)

product_id       0
date             0
sales            0
stock            1
price            1
promo_type_1     1
promo_type_2     1
cluster_id       1
hierarchy1_id    1
storetype_id     1
store_size       1
season           1
holiday          1
week             1
month_name       1
dtype: int64

In [12]:
#df_weekly = df
# select drop columns accordingly
drop_columns = ['stock','hierarchy1_id', 'storetype_id',  'store_size', 'promo_type_1', 'promo_type_2', 'week', 'month_name', 'holiday', 'season', 'cluster_id']
#drop_columns = ['hierarchy1_id', 'storetype_id',  'store_size', 'promo_type_2', 'week', 'month_name']
#df_weekly = df_week.drop(columns = drop_columns) #for all products
df_weekly = df_select.drop(columns = drop_columns) # for selected products

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 14, Finished, Available, Finished)

**Arima without encoder (for non categorical variables only, (price))**

In [13]:
# Adjust column names 
df_weekly = df_weekly.rename(columns={
    'product_id': 'unique_id',
    'date': 'ds',
    'sales': 'y'
})


def perform_forecasting(df_weekly, unique_id):


    filter_df = df_weekly[df_weekly['unique_id'] == unique_id].copy()

    # Extract the last date in the dataset
    last_date = filter_df['ds'].max()


    # Separate sales and date from exogeneous variables

    # Selecting and renaming the columns for the first DataFrame
    Y_ts = filter_df[['unique_id', 'ds', 'y']]

    # Selecting and renaming the columns for the second DataFrame
    X_ts = filter_df.drop(columns=['y'])
    X_ts['unique_id'] = X_ts.unique_id.astype(str)

    # Extract dates for train and test set 
    dates = Y_ts['ds'].unique()
    dtrain = dates[:-2] #2 weeks
    dtest = dates[-2:]

    Y_train = Y_ts.query('ds in @dtrain')
    Y_test = Y_ts.query('ds in @dtest') 

    X_train = X_ts.query('ds in @dtrain') 
    X_test = X_ts.query('ds in @dtest')


    #insert encode

    # Calculate median prices for each unique_id in X_train
    median_prices_1 = X_train.groupby('unique_id')['price'].transform('median')
    median_prices_2 = X_test.groupby('unique_id')['price'].transform('median')
    median_prices_3 = X_ts.groupby('unique_id')['price'].transform('median')


    # Find unique_ids with NaN median prices
    ids_with_nan_median_1 = X_train['unique_id'][median_prices_1.isnull()].unique()
    ids_with_nan_median_2 = X_test['unique_id'][median_prices_2.isnull()].unique()
    ids_with_nan_median_3 = X_ts['unique_id'][median_prices_3.isnull()].unique()


    # Fill NaN values in 'price' column with median prices (except for those with NaN median)
    X_train['price'].fillna(median_prices_1, inplace=True)
    X_test['price'].fillna(median_prices_2, inplace=True)
    X_ts['price'].fillna(median_prices_3, inplace=True)


    '''X_train['stock'] = X_train['stock'].fillna(method='ffill')
    X_test['stock'] = X_test['stock'].fillna(method='ffill')
    X_ts['stock'] = X_ts['stock'].fillna(method='ffill')'''


    # Fill NaN values for specific unique_ids with a default value (e.g., 0 or another appropriate value)
    default_price = 0  # Change this to your desired default value
    X_train.loc[X_train['unique_id'].isin(ids_with_nan_median_1) & X_train['price'].isnull(), 'price'] = default_price
    X_test.loc[X_test['unique_id'].isin(ids_with_nan_median_2) & X_test['price'].isnull(), 'price'] = default_price
    X_ts.loc[X_test['unique_id'].isin(ids_with_nan_median_3) & X_ts['price'].isnull(), 'price'] = default_price



    # Merge Y_train with X_ts to create the training dataset
    train = pd.merge(Y_train, X_ts, on=['unique_id', 'ds'], how='left')


    # Define models to use
    models = [AutoARIMA(season_length=52)] # represents weekly data

    # Define statsforecast object
    sf = StatsForecast(
        models=models, 
        freq='W-MON', # weekly frequency
        n_jobs=-1,
    )

    # Define horizon and level for AutoARIMA
    horizon = 2
    level = [95]

    # Forecast
    fcst = sf.forecast(df=train, h=horizon, X_df=X_test, level=level)
    fcst.head()

    # Extract predictions
    autoarima_predictions = fcst['AutoARIMA'].values

    # Evaluate the accuracy
    res = pd.merge(Y_test, fcst, on=['unique_id', 'ds'], how='left')

    # Calculate metrics
    mae = abs(res['y'] - res['AutoARIMA']).mean()
    mse = mean_squared_error(res['y'], res['AutoARIMA'])  # Using sklearn.metrics for MSE
    rmse = mse ** 0.5
    epsilon = 1e-10  # Small constant to avoid division by zero
    mape = (abs(res['y'] - res['AutoARIMA']) / (res['y'] + epsilon)).mean() * 100

    absolute_errors = abs(res['y'] - res['AutoARIMA'])
    wmape = (absolute_errors.sum() / abs(res['y']).sum()) * 100

    # Prepare future X_df
    future_dates = pd.date_range(start=last_date + pd.DateOffset(weeks=1), periods=horizon, freq='W-MON')

        # Find the last valid price for the specific unique_id in X_ts
    last_valid_price = X_ts['price'][(X_ts['unique_id'] == unique_id) & X_ts['price'].notna() & 
                                     (X_ts['price'] != float('inf')) & (X_ts['price'] != float('-inf'))].iloc[-1]

    future_X_df = pd.DataFrame({
        'unique_id': unique_id,
        'ds': future_dates,
        'price': last_valid_price
    })

    # Merge Y_ts with X_ts to create the training dataset
    predict_df = pd.merge(Y_ts, X_ts, on=['unique_id', 'ds'], how='left')

    # Train on entire dataset including the last 2 weeks
    fcst_final = sf.forecast(df=predict_df, h=horizon, X_df = future_X_df, level = level)



    # Return results 
    return {
        'unique_id': unique_id,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'WMAPE':wmape,
        'AutoARIMA_Predictions': autoarima_predictions,
        'Final_Predictions': fcst_final['AutoARIMA'].values

    }


# Loop through each product and perform forecasting
unique_ids = df_weekly['unique_id'].unique()  # Get all unique_ids
results_list = []


for unique_id in unique_ids:
    results = perform_forecasting(df_weekly, unique_id)
    results_list.append(results)
    
    
    
# Convert results to a DataFrame
results_df = pd.DataFrame(results_list)
print(results_df)



StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 15, Finished, Available, Finished)

  unique_id        MAE         MSE       RMSE        MAPE       WMAPE  \
0     P0103  23.596813  717.549922  26.787122  165.512886   94.387253   
1     P0131  14.876328  240.544339  15.509492   26.770668   26.098820   
2     P0569  14.128282  217.914838  14.761939  514.892673  256.877847   

    AutoARIMA_Predictions       Final_Predictions  
0  [48.275154, 48.918472]  [41.744858, 43.458923]  
1   [71.490074, 72.26258]    [58.94537, 61.15419]  
2   [18.849672, 20.40689]  [14.480704, 18.657902]  


In [14]:
results = pd.DataFrame(results_df)
results

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 16, Finished, Available, Finished)

,unique_id,MAE,MSE,RMSE,MAPE,WMAPE,AutoARIMA_Predictions,Final_Predictions
0,P0103,23.596813,717.549922,26.787122,165.512886,94.387253,"[48.275154, 48.918472]","[41.744858, 43.458923]"
1,P0131,14.876328,240.544339,15.509492,26.770668,26.098820,"[71.490074, 72.26258]","[58.94537, 61.15419]"
2,P0569,14.128282,217.914838,14.761939,514.892673,256.877847,"[18.849672, 20.40689]","[14.480704, 18.657902]"


In [15]:
#Saving csv
metrics_path = 'abfss://Projeto_ii_ML_models@onelake.dfs.fabric.microsoft.com/Konya_sales.Lakehouse/Files/W_exog vs Wout_exog/Exog_S0142_metrics_valid_forecast_price.csv'
results.to_csv(metrics_path, index=False)

StatementMeta(, dd46e904-7fe1-419f-a20d-f825455eb6db, 17, Finished, Available, Finished)